In [ ]:
#code to read all the apk's in the dir
from os import listdir
from os.path import isfile, join


apk_path = 'APKs'
#apk_path = '6test'
#sesName = apk_path + ".ag"
useSes = False

apks = []
for f in listdir(apk_path):
    if isfile(join(apk_path, f)) and f.endswith(".apk"):
        full = './' + apk_path + '/' + f
        apks.append(full)
        print("added: " + full)  
        

#apks = [f for f in listdir(apk_path) if (isfile(join(apk_path, f)) and f.endswith(".apk"))]

print("number apk's found: ", len(apks))

In [ ]:
#code to cache the analysis of the apk's (if this is enabled)
from androguard import misc
from androguard import session
from os import listdir
from os.path import isfile, join

if useSes:
    s = misc.get_default_session()
    for apk in apks:    
        digest = s.add(apk)
        print("analysed: " + apk)
else:
    print("not using session analysis cache");


In [ ]:
# code that uses get_xref_from() to see from where the code is called 
from androguard.core.analysis.analysis import ExternalMethod

def analyseMethod(ma, level=0):    
    #print(repr(ma))
    for pca, pm, offset in ma.get_xref_from(): #(ClassAnalysis, EncodedMethod or ExternalMethod, int
        print("L{} Call from {} => {}:{}".format(level, pca.name, pm.name, offset))
        

for apk in apks:
    print("*** " + apk )
    if(useSes):
        a, d, dx = s.get_objects_apk(filename = apk)
    else:
        a, d, dx = misc.AnalyzeAPK(apk)        
        
    #for ma in dx.find_methods(classname="Landroid/webkit/WebSettings;", methodname='setUserAgentString'):
    for ma in dx.find_methods(methodname='setUserAgentString'):
        analyseMethod(ma)
    

    

In [ ]:
#code using the get_call_graph() function from the androguard API
from networkx import nx
from networkx import MultiDiGraph
import matplotlib.pyplot as plt

 
cnt = 0
for apk in apks:
    cnt+=1
    if(useSes):
        a, d, dx = s.get_objects_apk(filename = apk)
    else:
        a, d, dx = misc.AnalyzeAPK(apk)        
        
    cg = dx.get_call_graph(methodname='setUserAgentString')    
    cg.name = apk
    print(nx.info(cg) + "\n") 
 

In [ ]:
#block that draws call-tree 
from networkx import nx
from networkx import MultiDiGraph
import matplotlib.pyplot as plt

 
cnt = 0
for apk in apks:
    cnt+=1
    if(useSes):
        a, d, dx = s.get_objects_apk(filename = apk)
    else:
        a, d, dx = misc.AnalyzeAPK(apk)            
    plt.figure(cnt, figsize=(8, 6))
    cg = dx.get_call_graph(methodname='setUserAgentString')    
    cg.name = apk
    print(nx.info(cg))
    pos = nx.spring_layout(cg) # circular_layout random_layout shell_layout spring_layout spectral_layout    
    nx.draw_networkx(cg)     
    
    
plt.show()    
